# SOU Exploratory Analyis & File Creation

### Libraries

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pdfplumber
import re

# Personal Lib for this project
from tools import *

### Testing on One File
- TODO:
    Compile everything into one function

In [66]:
# Path to for rep file
pdf_path = "../data/pdf/March20.pdf"

with pdfplumber.open(pdf_path, pages = [1]) as pdf:
    # Extract text from each page
    text = ""
    for page in pdf.pages:
        text += page.extract_text();

# Grab the text and figure out use case

In [91]:
def merge_and_pop(lst:list):
    '''
    merges and pops the name
    '''
    print(lst[1:3])
    if ',' in lst[1]:
        last, first = lst[1].lower(), lst[2].lower()
        lst[1] = first.capitalize() + ' ' + last[:-1].capitalize()
    else:
        lst[1] = lst[1] + ' ' + lst[2]
    lst.pop(2)
    return lst


def folder_to_csv(folder_path:str, test:bool = False):
    
    if test:
        folder = [folder_path]
    else:
        folder = folder_to_list(folder_path)

    i = 0 # used to track the number of files

    for file in folder:
        print(i, file)
        with pdfplumber.open(file, pages= [1]) as pdf:
            
            # Extract text from each page
            text = ""
            for page in pdf.pages:
                text += page.extract_text();
        if test:
            print(repr(text))

        matches = re.findall(r'Southern Oregon \d+..(#[\s\S]+?..TM T)', text, re.DOTALL)
        matches_date = re.findall(r'Date: \s*(........)', text)

        stats = matches[0]
        stats = stats.strip().split(sep = '\n')
        split_stats = list(map(str.split, stats))

        opponent = get_opponent(text)
        title = split_stats[0] + ['Opponent'] + ['Date']

        players = split_stats[1:-1]
        players = list(map(merge_and_pop, players))
        players = list(map(non_start, players))
        players

        if i == 0:
            stat_dict = {key: [] for key in title}

        for player in players:

            for stat, data in zip(title, player):
                stat_dict[stat].append(data)
            # stat_dict['Opponent'].append(opponent)#new
            # stat_dict['Date'].append(matches_date[0])#new
        for name in split_stats[1:-1]:
            stat_dict['Opponent'].append(opponent)
            stat_dict['Date'].append(matches_date[0])
 
        if len(stat_dict['PTS']) != len(stat_dict['Player']):
            print(matches_date, 'Length of PTS and Player do not match')
            #return
        i += 1

    return (stat_dict)

# Testing Area

In [92]:
len(folder_to_list('../data/pdf/'))

35

In [93]:
#x = folder_to_csv('../data/pdf')
x = folder_to_csv('../data/pdf/Dec14.pdf', test = True)

0 ../data/pdf/Dec14.pdf
'Southern Oregon (9-0, 3-0 CCC) -vs- Oregon Tech (6-3, 2-1 CCC)\n12/14/24 at Klamath Falls, Ore. (Danny Miles Court)\nDate: 12/14/24 Score By Period 1 2 3 4 Total\nTime: 5:30 PM\nSouthern Oregon 5 13 20 27 65\nAttendance: 1,210\nSite: Klamath Falls, Ore. (Danny Miles Court) Oregon Tech 19 12 13 14 58\nSouthern Oregon 65\n# Player GS MIN FG 3PT FT ORB-DRB REB PF A TO BLK STL PTS\n10 MCINTYRE,BRIDGETTE * 35 4-9 4-7 2-2 0-4 4 3 3 1 0 0 14\n12 SCHMERBACH,EMMA * 32 5-11 0-1 3-3 2-5 7 2 3 3 3 1 13\n20 BAIRD,MORGAN * 28 5-9 0-1 0-2 1-2 3 2 3 2 2 1 10\n01 MCINTYRE,MEGHAN * 26 2-6 0-1 3-6 0-3 3 2 3 5 0 1 7\n23 ROBBINS,CLARA * 13 1-1 0-0 0-0 0-5 5 2 0 1 1 1 2\n32 DIGIULIO,ELIZA 30 2-5 1-2 4-4 2-2 4 2 0 1 0 2 9\n33 WILLIAMS,MALLORY 5 2-2 2-2 0-0 0-1 1 1 0 0 0 0 6\n13 WRIGHT,KEELEY 10 1-1 0-0 2-3 1-2 3 0 1 3 0 0 4\n22 SCHEPPELE,SIERRA 12 0-1 0-1 0-0 0-2 2 0 0 0 0 0 0\n05 SCURLOCK,MAKAYLA 9 0-2 0-2 0-0 0-1 1 1 1 0 0 0 0\nTM TEAM 0 0-0 0-0 0-0 0-1 1 0 0 1 0 0 0\nTotals - 200 

In [84]:
for key in x:
    print(key, len(x[key]))

# 10
Player 10
GS 10
MIN 10
FG 10
3PT 10
FT 10
ORB-DRB 10
REB 10
PF 10
A 10
TO 10
BLK 10
STL 10
PTS 5
Opponent 10
Date 10


In [94]:
x['GS']

['-', '-', '-', '-', '-', '-', '-', '-', '-', '-']

In [71]:
pd.DataFrame.from_dict(x)

ValueError: All arrays must be of the same length